# Model Performance Transformations

Lets practice some basic data transformation for ML performance enhancement

In [113]:
# Imports

import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [114]:
# Categorical data analyser

def cat_var(df, cols):
    '''
    Return: a Pandas dataframe object with the following columns:
        - "categorical_variable" => every categorical variable include as an input parameter (string).
        - "number_of_possible_values" => the amount of unique values that can take a given categorical variable (integer).
        - "values" => a list with the posible unique values for every categorical variable (list).

    Input parameters:
        - df -> Pandas dataframe object: a dataframe with categorical variables.
        - cols -> list object: a list with the name (string) of every categorical variable to analyse.
    '''
    cat_list = []
    for col in cols:
        cat = df[col].unique()
        cat_num = len(cat)
        cat_dict = {"categorical_variable":col,
                    "number_of_possible_values":cat_num,
                    "values":cat}
        cat_list.append(cat_dict)
    df = pd.DataFrame(cat_list).sort_values(by="number_of_possible_values", ascending=False)
    return df.reset_index(drop=True)

## Scaling

Some ML algorithms have problems performing well whenever the data scale differ greatly between features. In those cases scaling the data is your best option.

- [RobustScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler)

- [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler)

Try both options and see what happens with performance (i.e.: AUC).

<img src="../images/scaling.png" alt="Drawing" style="width: 500px;"/>

In [115]:
# Weather dataset (https://www.kaggle.com/jsphyg/weather-dataset-rattle-package)

weather = pd.read_csv('../data/weatherAUS.csv')
print(weather.shape)
weather.head()

(145460, 23)


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [116]:
# Uluru weather (numerical features)

weather = weather[weather['Location'].isin(['Uluru'])].reset_index(drop=True)
weather = weather[weather['RainToday'].isin(['No','Yes'])].reset_index(drop=True)
weather = weather[weather['RainTomorrow'].isin(['No','Yes'])]
weather = weather[['MinTemp',
                   'MaxTemp',
                   'Rainfall',
                   'WindSpeed9am',
                   'WindSpeed3pm',
                   'Humidity9am',
                   'Humidity3pm',
                   'Pressure9am',
                   'Pressure3pm',
                   'Temp9am',
                   'Temp3pm',
                   'RainTomorrow']]
weather = weather.dropna().reset_index(drop=True)
col_weather = list(weather.columns)
print(col_weather)
print(weather.shape)
print(weather.describe())
weather.head()

['MinTemp', 'MaxTemp', 'Rainfall', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Temp9am', 'Temp3pm', 'RainTomorrow']
(1479, 12)
           MinTemp      MaxTemp     Rainfall  WindSpeed9am  WindSpeed3pm  \
count  1479.000000  1479.000000  1479.000000   1479.000000   1479.000000   
mean     14.368627    30.402299     0.716700     17.613928     17.050710   
std       7.432857     7.624058     4.208585      7.887082      6.893016   
min      -1.900000    11.300000     0.000000      0.000000      0.000000   
25%       8.100000    23.800000     0.000000     11.000000     11.000000   
50%      14.900000    31.200000     0.000000     17.000000     17.000000   
75%      20.800000    37.100000     0.000000     24.000000     22.000000   
max      31.000000    44.400000    83.800000     41.000000     48.000000   

       Humidity9am  Humidity3pm  Pressure9am  Pressure3pm      Temp9am  \
count  1479.000000  1479.000000  1479.000000  1479.000000  1479.0

,MinTemp,MaxTemp,Rainfall,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainTomorrow
0,19.7,30.0,0.8,30.0,24.0,76.0,54.0,1010.6,1007.5,21.7,28.4,No
1,21.6,33.1,0.0,22.0,11.0,44.0,33.0,1010.5,1006.5,24.6,31.3,No
2,21.3,36.1,0.0,24.0,13.0,39.0,27.0,1006.9,1002.7,27.6,34.5,No
3,22.9,37.7,0.0,28.0,13.0,35.0,22.0,1006.0,1002.1,28.7,35.4,No
4,24.0,39.0,0.0,20.0,19.0,33.0,21.0,1006.9,1003.5,29.9,37.3,No


In [117]:
# Features + target

X = weather[['MinTemp',
          'MaxTemp',
          'Rainfall',
          'WindSpeed9am',
          'WindSpeed3pm',
          'Humidity9am',
          'Humidity3pm',
          'Pressure9am',
          'Pressure3pm',
          'Temp9am',
          'Temp3pm']]
y = pd.get_dummies(weather['RainTomorrow'], drop_first=True)['Yes']
print(X.shape,y.shape)

(1479, 11) (1479,)


In [118]:
# Train + test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}")
print(f"X_train: {type(X_train)}, X_test: {type(X_test)}, y_train: {type(y_train)}, y_test: {type(y_test)}")

X_train: (1183, 11), X_test: (296, 11), y_train: (1183,), y_test: (296,)
X_train: <class 'pandas.core.frame.DataFrame'>, X_test: <class 'pandas.core.frame.DataFrame'>, y_train: <class 'pandas.core.series.Series'>, y_test: <class 'pandas.core.series.Series'>


In [119]:
# Scaling


scaler = RobustScaler()
scaled_data = scaler.fit_transform(X_train)
scaled_data

scaled_df = pd.DataFrame(scaled_data)
scaled_df




,0,1,2,3,4,5,6,7,8,9,10
0,-0.535433,-0.785185,0.0,0.076923,0.636364,0.84375,1.3750,0.948454,1.072917,-0.632727,-0.824903
1,-1.094488,-0.851852,0.0,-0.769231,-0.545455,0.68750,0.1875,0.731959,0.802083,-0.858182,-0.871595
2,0.984252,0.874074,0.0,-0.769231,0.000000,-0.46875,-0.3750,-1.134021,-1.177083,0.960000,0.778210
3,-1.086614,-0.881481,0.0,0.000000,0.181818,0.62500,0.5000,0.731959,0.729167,-0.836364,-0.964981
4,0.401575,0.533333,0.0,1.076923,0.454545,-0.81250,-0.7500,-0.762887,-0.500000,0.785455,0.513619
...,...,...,...,...,...,...,...,...,...,...,...
1178,-0.488189,-0.244444,0.0,-0.769231,2.181818,0.59375,0.3125,-0.175258,-0.322917,-0.370909,-0.163424
1179,0.677165,0.466667,0.0,-0.153846,-0.363636,-0.06250,0.2500,-0.917526,-0.864583,0.676364,0.505837
1180,-0.677165,-0.148148,0.0,-0.461538,-1.000000,0.00000,-0.3750,0.484536,0.312500,-0.385455,-0.186770
1181,-0.779528,-0.770370,0.0,-0.153846,0.636364,0.00000,-0.5625,1.030928,1.114583,-0.807273,-0.770428


In [120]:
scaler = RobustScaler()
scaled_data = scaler.fit_transform(X_test)
scaled_data

scaled_df = pd.DataFrame(scaled_data,
                        )
scaled_df

,0,1,2,3,4,5,6,7,8,9,10
0,0.730924,0.877339,0.0,0.230769,0.222222,-0.645161,-0.473684,-0.572271,-0.607450,1.017682,0.900862
1,0.827309,0.819127,0.2,0.846154,-0.666667,-0.419355,-0.157895,-0.820059,-0.871060,0.884086,0.780172
2,-0.441767,-0.045738,0.0,-0.846154,-0.222222,0.580645,0.263158,0.135693,-0.011461,-0.247544,0.012931
3,-0.208835,0.794179,0.0,-0.846154,-0.222222,-0.903226,-0.789474,-0.796460,-0.561605,0.310413,0.685345
4,0.971888,0.062370,0.0,-0.307692,1.000000,-0.677419,1.052632,-0.171091,0.022923,0.569745,-0.168103
...,...,...,...,...,...,...,...,...,...,...,...
291,0.851406,0.586279,0.0,0.153846,0.333333,0.000000,0.526316,-1.233038,-1.088825,0.797642,0.478448
292,-1.036145,-0.860707,0.0,0.000000,0.222222,0.548387,0.631579,1.162242,1.088825,-0.876228,-0.857759
293,0.602410,-0.220374,0.0,-0.307692,0.000000,1.322581,2.631579,-0.253687,-0.057307,0.168959,-0.280172
294,0.056225,-0.644491,0.0,1.384615,0.555556,0.000000,0.578947,-0.690265,0.011461,-0.011788,-0.840517


In [121]:
# Linear model

linear_model = LogisticRegression(max_iter=1000)
linear_param = linear_model.fit(X_train, y_train)
linear_pred = linear_model.predict(X_test)
linear_auc = roc_auc_score(y_test, linear_pred)
print(f"Linear model AUC is: {linear_auc}")

Linear model AUC is: 0.7278168345050351


In [122]:
# Ensemble model

ensemble_model = RandomForestClassifier()
ensemble_param = ensemble_model.fit(X_train, y_train)
ensemble_pred = ensemble_model.predict(X_test)
ensemble_auc = roc_auc_score(y_test, ensemble_pred)
print(f"Linear model AUC is: {ensemble_auc}")

Linear model AUC is: 0.6715751472544176


---

## Enconding

ML algorithms do not support categorical data. Therefore you need to find a way to transform categorical data into numerical. You must compare the results using both techniques: __One Hot Encoding__ or __Label Encoding__

- [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder)

- [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder)

<img src="../images/encoding.png" alt="Drawing" style="width: 500px;"/>

In [123]:
# Mushrooms dataset (https://www.kaggle.com/uciml/mushroom-classification)
mushrooms.info()
mushrooms = pd.read_csv('../data/mushrooms.csv')
col_mushrooms = list(mushrooms.columns)
print(mushrooms.shape)
mushrooms.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [124]:
# Features analysis

cat_mushrooms = cat_var(mushrooms, col_mushrooms)
cat_mushrooms


,categorical_variable,number_of_possible_values,values
0,gill-color,12,"[k, n, g, p, w, h, u, e, b, r, y, o]"
1,cap-color,10,"[n, y, w, g, e, p, b, u, c, r]"
2,spore-print-color,9,"[k, n, u, h, w, r, o, y, b]"
3,odor,9,"[p, a, l, n, f, c, y, s, m]"
4,stalk-color-below-ring,9,"[w, p, g, b, n, e, y, o, c]"
5,stalk-color-above-ring,9,"[w, g, p, n, b, e, o, c, y]"
6,habitat,7,"[u, g, m, d, p, w, l]"
7,cap-shape,6,"[x, b, s, f, k, c]"
8,population,6,"[s, n, a, v, y, c]"
9,ring-type,5,"[p, e, l, f, n]"


In [125]:
# Features + target (encoding). IMPORTANT: you may pick any of the 2-labeled features as you target (choose wisely!!!)
cat_mushrooms['categorical_variable'].unique
#cat_mushrooms['values'].unique
encoding_class = {'p':1, 'e':2}
def ordinal_encoding(x):
    for key in encoding_class:
        if x == key:
            return encoding_class[key]


mushrooms['class_num'] = mushrooms['class'].apply(ordinal_encoding)
mushrooms





,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class_num
0,p,x,s,n,t,p,f,c,n,k,...,w,w,p,w,o,p,k,s,u,1
1,e,x,s,y,t,a,f,c,b,k,...,w,w,p,w,o,p,n,n,g,2
2,e,b,s,w,t,l,f,c,b,n,...,w,w,p,w,o,p,n,n,m,2
3,p,x,y,w,t,p,f,c,n,n,...,w,w,p,w,o,p,k,s,u,1
4,e,x,s,g,f,n,f,w,b,k,...,w,w,p,w,o,e,n,a,g,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,o,o,p,o,o,p,b,c,l,2
8120,e,x,s,n,f,n,a,c,b,y,...,o,o,p,n,o,p,b,v,l,2
8121,e,f,s,n,f,n,a,c,b,n,...,o,o,p,o,o,p,b,c,l,2
8122,p,k,y,n,f,y,f,c,n,b,...,w,w,p,w,o,e,w,v,l,1


In [126]:
mushrooms.isnull().sum()
mushrooms.dropna()
mushrooms.iloc[1]

class                       e
cap-shape                   x
cap-surface                 s
cap-color                   y
bruises                     t
odor                        a
gill-attachment             f
gill-spacing                c
gill-size                   b
gill-color                  k
stalk-shape                 e
stalk-root                  c
stalk-surface-above-ring    s
stalk-surface-below-ring    s
stalk-color-above-ring      w
stalk-color-below-ring      w
veil-type                   p
veil-color                  w
ring-number                 o
ring-type                   p
spore-print-color           n
population                  n
habitat                     g
class_num                   2
Name: 1, dtype: object

In [127]:
# One hot Encoding
cat_cols = ['class', 'cap-shape', 'spore-print-color', 
            'cap-shape', 'cap-surface', 'cap-color','bruises', 'odor', 'gill-attachment',
            'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root',
            'stalk-surface-above-ring' ,'stalk-surface-below-ring','stalk-color-above-ring', 'stalk-color-below-ring',
            'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color',
            'population', 'habitat'] 

mushrooms_one_hot_encoding = pd.get_dummies(mushrooms[cat_cols], 
                                          drop_first=True)
mushrooms_one_hot_encoding

,class_p,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,spore-print-color_h,spore-print-color_k,spore-print-color_n,spore-print-color_o,...,population_n,population_s,population_v,population_y,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,1,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
1,0,0,0,0,0,1,0,0,1,0,...,1,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,1,0,0,0
3,1,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8120,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
8121,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8122,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0


In [128]:
X=[[mushrooms_one_hot_encoding]]
y=[mushrooms['class_num']]

In [129]:
# Scaling
# Scaling Standard
scaler = StandardScaler()
scaled_data = scaler.fit_transform(mushrooms_one_hot_encoding)
#scaled_data = scaler.fit_transform(X)
scaled_data


array([[ 1.0366127 , -0.02219484, -0.79620985, ..., -0.40484176,
         4.59086996, -0.15558197],
       [-0.96468045, -0.02219484, -0.79620985, ..., -0.40484176,
        -0.21782364, -0.15558197],
       [-0.96468045, -0.02219484, -0.79620985, ..., -0.40484176,
        -0.21782364, -0.15558197],
       ...,
       [-0.96468045, -0.02219484,  1.2559503 , ..., -0.40484176,
        -0.21782364, -0.15558197],
       [ 1.0366127 , -0.02219484, -0.79620985, ..., -0.40484176,
        -0.21782364, -0.15558197],
       [-0.96468045, -0.02219484, -0.79620985, ..., -0.40484176,
        -0.21782364, -0.15558197]])

In [130]:
df_X=pd.DataFrame(scaled_data)
df_X

,0,1,2,3,4,5,6,7,8,9,...,99,100,101,102,103,104,105,106,107,108
0,1.036613,-0.022195,-0.79621,-0.336878,-0.062885,1.105487,-0.501384,1.827497,-0.565410,-0.077094,...,-0.227567,2.347257,-0.994599,-0.51672,-0.599531,-0.337783,-0.193088,-0.404842,4.590870,-0.155582
1,-0.964680,-0.022195,-0.79621,-0.336878,-0.062885,1.105487,-0.501384,-0.547197,1.768629,-0.077094,...,4.394315,-0.426029,-0.994599,-0.51672,1.667970,-0.337783,-0.193088,-0.404842,-0.217824,-0.155582
2,-0.964680,-0.022195,-0.79621,-0.336878,-0.062885,-0.904579,-0.501384,-0.547197,1.768629,-0.077094,...,4.394315,-0.426029,-0.994599,-0.51672,-0.599531,-0.337783,5.178988,-0.404842,-0.217824,-0.155582
3,1.036613,-0.022195,-0.79621,-0.336878,-0.062885,1.105487,-0.501384,1.827497,-0.565410,-0.077094,...,-0.227567,2.347257,-0.994599,-0.51672,-0.599531,-0.337783,-0.193088,-0.404842,4.590870,-0.155582
4,-0.964680,-0.022195,-0.79621,-0.336878,-0.062885,1.105487,-0.501384,-0.547197,1.768629,-0.077094,...,-0.227567,-0.426029,-0.994599,-0.51672,1.667970,-0.337783,-0.193088,-0.404842,-0.217824,-0.155582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,-0.964680,-0.022195,-0.79621,2.968433,-0.062885,-0.904579,-0.501384,-0.547197,-0.565410,-0.077094,...,-0.227567,-0.426029,-0.994599,-0.51672,-0.599531,2.960477,-0.193088,-0.404842,-0.217824,-0.155582
8120,-0.964680,-0.022195,-0.79621,-0.336878,-0.062885,1.105487,-0.501384,-0.547197,-0.565410,-0.077094,...,-0.227567,-0.426029,1.005431,-0.51672,-0.599531,2.960477,-0.193088,-0.404842,-0.217824,-0.155582
8121,-0.964680,-0.022195,1.25595,-0.336878,-0.062885,-0.904579,-0.501384,-0.547197,-0.565410,-0.077094,...,-0.227567,-0.426029,-0.994599,-0.51672,-0.599531,2.960477,-0.193088,-0.404842,-0.217824,-0.155582
8122,1.036613,-0.022195,-0.79621,2.968433,-0.062885,-0.904579,-0.501384,-0.547197,-0.565410,-0.077094,...,-0.227567,-0.426029,1.005431,-0.51672,-0.599531,2.960477,-0.193088,-0.404842,-0.217824,-0.155582


In [131]:
# Train + test
X_train, X_test, y_train, y_test = train_test_split(df_X, y, test_size=0.2, random_state=42)
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}")









ValueError: Found input variables with inconsistent numbers of samples: [8124, 1]

In [94]:
# Linear model

linear_model = LogisticRegression(max_iter=1000)
linear_param = linear_model.fit(X_train, y_train)
linear_pred = linear_model.predict(X_test)
linear_auc = roc_auc_score(y_test, linear_pred)
print(f"Linear model AUC is: {linear_auc}")

Linear model AUC is: 0.7278168345050351


In [95]:
# Ensemble model

ensemble_model = RandomForestClassifier()
ensemble_param = ensemble_model.fit(X_train, y_train)
ensemble_pred = ensemble_model.predict(X_test)
ensemble_auc = roc_auc_score(y_test, ensemble_pred)
print(f"Linear model AUC is: {ensemble_auc}")

Linear model AUC is: 0.6697700931027931


---

## Bonus

Now that you can grasp the potential of pre-processing your data...what would you do about the following dataset?

<img src="../images/bonus.jpg" alt="Drawing" style="width: 500px;"/>

In [81]:
# Netflix dataset (https://www.kaggle.com/shivamb/netflix-shows)

netflix = pd.read_csv('../data/netflix_titles.csv')
col_netflix = list(netflix.columns)
print(netflix.shape)
netflix.head()

(7787, 12)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [82]:
netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7787 entries, 0 to 7786
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7787 non-null   object
 1   type          7787 non-null   object
 2   title         7787 non-null   object
 3   director      5398 non-null   object
 4   cast          7069 non-null   object
 5   country       7280 non-null   object
 6   date_added    7777 non-null   object
 7   release_year  7787 non-null   int64 
 8   rating        7780 non-null   object
 9   duration      7787 non-null   object
 10  listed_in     7787 non-null   object
 11  description   7787 non-null   object
dtypes: int64(1), object(11)
memory usage: 730.2+ KB


In [ ]:
# ML workflow -> ¿what would you do?










---